<a id='_1'></a>
# 1 Введение 

<a id='_1_1'></a>
## 1.1 Постановка задачи

**Заказчик** -- интернет-магазин  
  
**Задача** -- предложить инструмент, который будет искать токсичные комментарии в описании товаров и отправлять данные описания на модерацию.  
В качестве инструмента использовать модель ML. Модель необходимо обучить классифицировать комментарии на позитивные и негативные.  

**Метрика качества модели** -- F1  
  
**Значение метрики качества модели** -- не менее 0.75

<a id='_1_2'></a>
## 1.2 Исходные данные

<a id='_1_2_1'></a>
### 1.2.1 Общее описание
Передан набор данных с разметкой о токсичности правок

<a id='_1_2_2'></a>
### 1.2.2 Переданные файлы
- toxic_comments.csv

<a id='_1_2_3'></a>
### 1.2.3 Описание данных
Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак

<a id='_1_3'></a>
## 1.3 План обработки и анализа данных

### 1.3.1 Предобработка данных
- обзор данных;
- очистка текста;
- лемматизация текста

### 1.3.2 Обучение моделей ML

#### 1.3.2.2 Разделение набора данных на выборки

#### 1.3.2.2 Логистическая регрессия

#### 1.3.2.3 Случайный лес

### 1.3.3 Вывод

<a id='_1_4'></a>
## 1.4 Пользовательские функции

In [1]:
import numpy as np
from numpy.random import RandomState
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import HTML
import re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

from pymystem3 import Mystem
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_rows = 10
#pd.options.display.max_columns = 50
pd.options.mode.chained_assignment = None

<a id='_2'></a>
# 2 Основная часть

## 2.1 Предобработка данных

**Обзор данных**

In [2]:
data = pd.read_csv('toxic_comments.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
data.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


**Очистка текста**

In [5]:
def clear_text(text):
    return " ".join(re.sub(r"[^a-zA-Z']", ' ', text).split())

In [6]:
data['text'] = data['text'].apply(clear_text)
data.head(10)

,text,toxic
0,Explanation Why the edits made under my userna...,0
1,D'aww He matches this background colour I'm se...,0
2,Hey man I'm really not trying to edit war It's...,0
3,More I can't make any real suggestions on impr...,0
4,You sir are my hero Any chance you remember wh...,0
5,Congratulations from me as well use the tools ...,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


**Приведение слов текстов к нижнему регистру**

In [7]:
data['text'] = data['text'].str.lower()

**Лемматизация текста**

In [8]:
m = Mystem()

def lemmatize(text):
    return "".join(m.lemmatize(text))

In [9]:
%%time
lemmatize(data.loc[0, 'text'])

CPU times: user 11.8 ms, sys: 12.3 ms, total: 24.1 ms
Wall time: 910 ms


"explanation why the edits made under my username hardcore metallica fan were reverted they weren't vandalisms just closure on some gas after i voted at new york dolls fac and please don't remove the template from the talk page since i'm retired now\n"

Лемматизация всего набора данных на имеющихся вычислительных мощностях займёт порядка 1,5 дня. Ограничим выборку 80 000 значениями

In [10]:
state = RandomState(1457)

txt_corpus = data.sample(n=80000, replace=False, random_state=state)

In [11]:
%%time

for i in txt_corpus.index:
    txt_corpus.loc[i, 'text_lemm'] = lemmatize(txt_corpus.loc[i, 'text'])

txt_corpus.head(10)

CPU times: user 8min 38s, sys: 9.37 s, total: 8min 48s
Wall time: 9min 37s


,text,toxic,text_lemm
157809,i could just give you an email address that i ...,0,i could just give you an email address that i ...
47841,i totally understand if you ever become intere...,0,i totally understand if you ever become intere...
131316,in so far the list shoudlnt be used in the def...,0,in so far the list shoudlnt be used in the def...
35364,hey i didn't come up with that native australi...,1,hey i didn't come up with that native australi...
122849,you got someone banned who would want to fool ...,1,you got someone banned who would want to fool ...
154274,re cv sorry no copyright violation was made th...,0,re cv sorry no copyright violation was made th...
83254,does not dicuss any work towards a universal d...,0,does not dicuss any work towards a universal d...
38465,december utc pathoschild i could kiss you a th...,0,december utc pathoschild i could kiss you a th...
44648,yeah when you read the black book of buried se...,0,yeah when you read the black book of buried se...
42531,see you are just so cute and i can work with y...,0,see you are just so cute and i can work with y...


***Промежуточный итог***  
  
В ходе предобработки данных:
- была проведена очистка текстов;
- слова текстов приведены к нижнему регистру;
- была проведена лемматизация слов текстов, при этом, в силу больших временных затрат на леммитизацию всего набора, было принято решение о выделии набора размером 80 000 текстов и лемматизация была проведена для него. 

## 2.2 Обучение моделей ML

### 2.2.1 Разделение набора данных на выборки

In [12]:
features = txt_corpus['text_lemm']
target = txt_corpus['toxic']

In [13]:
features_train, features_valid, target_train, target_valid = train_test_split(features,
                                                                              target, test_size=0.2, random_state=1457)

In [14]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

features_train = count_tf_idf.fit_transform(features_train)
features_valid = count_tf_idf.transform(features_valid)

In [16]:
print('Количество "1": {:.2%}'.format(target_train.sum() / len(target_train)))

Количество "1": 10.10%


Классы целевого признака не сбалансированы. Этот факт необходимо учитывать при построении моделей ML

### 2.2.2 Логистическая регрессия

In [17]:
%%time
logreg_model = LogisticRegression(class_weight = 'balanced', #solver='liblinear', 
                                  random_state=1457)

param_grid = {
    'C': [2, 4, 6],
    'class_weight': ['balanced', None],
    'penalty': ['l1', 'l2'],
}

CV_result = GridSearchCV(estimator=logreg_model, param_grid=param_grid, scoring='f1', cv=3)

CV_result.fit(features_train, target_train)

print(CV_result.best_params_)
print(CV_result.best_score_)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


{'C': 4, 'class_weight': 'balanced', 'penalty': 'l1'}
0.7549281756692047
CPU times: user 1min 58s, sys: 1min 19s, total: 3min 17s
Wall time: 3min 18s


In [18]:
logreg_model = LogisticRegression(C=4, 
                                  class_weight = 'balanced', 
                                  penalty='l1',
                                  random_state=1457)
logreg_model.fit(features_train, target_train)
print('F1:', f1_score(target_valid, logreg_model.predict(features_valid)))

F1: 0.7511792452830188


In [19]:
logreg_model = LogisticRegression(C=4, 
                                  #class_weight = 'balanced', 
                                  penalty='l1',
                                  random_state=1457)
logreg_model.fit(features_train, target_train)
print('F1:', f1_score(target_valid, logreg_model.predict(features_valid)))

F1: 0.7763975155279503


### 2.2.3 Случайный лес

In [20]:
%%time
random_forest_model = RandomForestClassifier(class_weight = 'balanced', random_state=1457)

param_grid = { 
    'n_estimators': [10, 100, 150],
    'max_depth' : [10, 50, 100]
}

CV_result = GridSearchCV(estimator=random_forest_model, param_grid=param_grid, scoring='f1', cv=3)

CV_result.fit(features_train, target_train)

print(CV_result.best_params_)
print(CV_result.best_score_)

{'max_depth': 100, 'n_estimators': 150}
0.5354062664608791
CPU times: user 22min 59s, sys: 0 ns, total: 22min 59s
Wall time: 23min 6s


In [21]:
random_forest_model = RandomForestClassifier(random_state = 1457, class_weight = 'balanced', max_depth=100, n_estimators=150)
random_forest_model.fit(features_train, target_train)
print('F1:', f1_score(target_valid, random_forest_model.predict(features_valid)))

F1: 0.5301080802882142


***Промежуточный итог***  
  
По итогам обучения моделей ML можно сделать следующие выводы:
- лучшее качество получено на модели логистической регрессии;

# 3 Вывод

В ходе выполнения работы было сделано следующее.  
  
В ходе предобработки данных:
- была проведена очистка текстов;
- слова текстов были приведены к нижнему регистру;
- была проведена лемматизация слов текстов, при этом, в силу больших временных затрат на леммитизацию всего набора, было принято решение о выделии набора размером 80 000 текстов и лемматизация была проведена для него.
  
По итогам обучения моделей ML были сделаны следующие выводы:
- лучшее качество получено на модели логистической регрессии;
  
***Таким образом, для предсказания токсичности комментариев в описании товаров необходимо использовать модель логистической регрессии.***